In [26]:
import csv
import time
import random
import requests
from bs4 import BeautifulSoup

"""Generate a url from job position and location search result"""
def get_url(position,location):
    template = 'https://www.indeed.com/jobs?q=title%3A({})&l={}&jt=fulltime&limit=50'
    url = template.format(position,location)
    return url

"""Extract job data from a single listing"""
def get_listing(card):
    sleeptime = random.randrange(3,6)
    time.sleep(sleeptime)
    atag = card.h2.a
    title = atag.get('title')
    company = card.find('span', 'company').text.strip()
    job_url = 'https://www.indeed.com/viewjob?jk=' + card.get('data-jk')
    response_desc = requests.get(job_url)
    soup_desc = BeautifulSoup(response_desc.text, 'html.parser')
    description = soup_desc.find('div', 'jobsearch-jobDescriptionText')
    if description is not None:
        description = description.text.strip().replace('\n', ' ')
    location = soup_desc.find('div', "jobsearch-InlineCompanyRating")
    if location is not None:
        location = location.find_all('div')[-1].text.strip()
    listing = (title, company, location, description, job_url)
    return listing

"""Run job search and save all results to csv file"""
def main(position, location):
    listings = []
    url = get_url(position, location)
    
    while True:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find_all('div', 'jobsearch-SerpJobCard')
        
        """Get all job listings from current search page"""
        for card in cards:
            listing = get_listing(card)
            listings.append(listing)
            
        """Check if reached last search page"""
        try:
            url = 'https://www.indeed.com' + soup.find('a', {'aria-label': 'Next'}).get('href')
        except AttributeError:
            break
    
    """Create a csv file with our search results"""
    csv_title = 'listings_{}.csv'.format(location.replace(" ",""))
    with open(csv_title, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Title', 'Company', 'Location', 'Description', 'URL'])
        writer.writerows(listings)

In [27]:
main('data scientist','united states')